<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

*  [Bag Documenation](http://dask.pydata.org/en/latest/bag.html)
*  [Bag API](http://dask.pydata.org/en/latest/bag-api.html)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [1]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [2]:
# each element is a text file, where each line is a JSON object
# note that the compression is handles automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Xavier", "transactions": [{"transaction-id": 6350, "amount": 149}, {"transaction-id": 6786, "amount": 118}, {"transaction-id": 8133, "amount": 132}, {"transaction-id": 10459, "amount": 135}, {"transaction-id": 10892, "amount": 127}, {"transaction-id": 13961, "amount": 145}, {"transaction-id": 14650, "amount": 136}, {"transaction-id": 17970, "amount": 155}, {"transaction-id": 21538, "amount": 115}, {"transaction-id": 22100, "amount": 129}, {"transaction-id": 24374, "amount": 147}, {"transaction-id": 24980, "amount": 130}, {"transaction-id": 26821, "amount": 127}, {"transaction-id": 26878, "amount": 127}, {"transaction-id": 28354, "amount": 132}, {"transaction-id": 32782, "amount": 129}, {"transaction-id": 32908, "amount": 124}, {"transaction-id": 33311, "amount": 131}, {"transaction-id": 34188, "amount": 120}, {"transaction-id": 36309, "amount": 120}, {"transaction-id": 37147, "amount": 113}, {"transaction-id": 41350, "amount": 146}, {"transaction-id": 45939, "amoun

In [3]:
# Requires `s3fs` library
# # each partition is a remote CSV text file
# b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv')
# b.take(1)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [4]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<map-lam..., npartitions=10>

In [5]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [6]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Xavier", "transactions": [{"transaction-id": 6350, "amount": 149}, {"transaction-id": 6786, "amount": 118}, {"transaction-id": 8133, "amount": 132}, {"transaction-id": 10459, "amount": 135}, {"transaction-id": 10892, "amount": 127}, {"transaction-id": 13961, "amount": 145}, {"transaction-id": 14650, "amount": 136}, {"transaction-id": 17970, "amount": 155}, {"transaction-id": 21538, "amount": 115}, {"transaction-id": 22100, "amount": 129}, {"transaction-id": 24374, "amount": 147}, {"transaction-id": 24980, "amount": 130}, {"transaction-id": 26821, "amount": 127}, {"transaction-id": 26878, "amount": 127}, {"transaction-id": 28354, "amount": 132}, {"transaction-id": 32782, "amount": 129}, {"transaction-id": 32908, "amount": 124}, {"transaction-id": 33311, "amount": 131}, {"transaction-id": 34188, "amount": 120}, {"transaction-id": 36309, "amount": 120}, {"transaction-id": 37147, "amount": 113}, {"transaction-id": 41350, "amount": 146}, {"transaction-id": 45939, "amoun

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [7]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Xavier',
  'transactions': [{'transaction-id': 6350, 'amount': 149},
   {'transaction-id': 6786, 'amount': 118},
   {'transaction-id': 8133, 'amount': 132},
   {'transaction-id': 10459, 'amount': 135},
   {'transaction-id': 10892, 'amount': 127},
   {'transaction-id': 13961, 'amount': 145},
   {'transaction-id': 14650, 'amount': 136},
   {'transaction-id': 17970, 'amount': 155},
   {'transaction-id': 21538, 'amount': 115},
   {'transaction-id': 22100, 'amount': 129},
   {'transaction-id': 24374, 'amount': 147},
   {'transaction-id': 24980, 'amount': 130},
   {'transaction-id': 26821, 'amount': 127},
   {'transaction-id': 26878, 'amount': 127},
   {'transaction-id': 28354, 'amount': 132},
   {'transaction-id': 32782, 'amount': 129},
   {'transaction-id': 32908, 'amount': 124},
   {'transaction-id': 33311, 'amount': 131},
   {'transaction-id': 34188, 'amount': 120},
   {'transaction-id': 36309, 'amount': 120},
   {'transaction-id': 37147, 'amount': 113},
   {'transa

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [8]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 48,
  'name': 'Alice',
  'transactions': [{'transaction-id': 1219, 'amount': 4428},
   {'transaction-id': 1261, 'amount': 4314},
   {'transaction-id': 1486, 'amount': 4442},
   {'transaction-id': 2124, 'amount': 3775},
   {'transaction-id': 3949, 'amount': 4194},
   {'transaction-id': 4079, 'amount': 3781},
   {'transaction-id': 4537, 'amount': 4365},
   {'transaction-id': 4800, 'amount': 4082},
   {'transaction-id': 5354, 'amount': 3880},
   {'transaction-id': 5519, 'amount': 4179},
   {'transaction-id': 5607, 'amount': 4885},
   {'transaction-id': 7020, 'amount': 4374},
   {'transaction-id': 7081, 'amount': 4056},
   {'transaction-id': 7442, 'amount': 3954},
   {'transaction-id': 8565, 'amount': 3954},
   {'transaction-id': 8625, 'amount': 4385},
   {'transaction-id': 10567, 'amount': 4290},
   {'transaction-id': 10595, 'amount': 3903},
   {'transaction-id': 12044, 'amount': 3656},
   {'transaction-id': 12232, 'amount': 4649},
   {'transaction-id': 12616, 'amount': 4274},
   

In [9]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 149},
 {'name': 'Alice', 'count': 169},
 {'name': 'Alice', 'count': 136},
 {'name': 'Alice', 'count': 273},
 {'name': 'Alice', 'count': 107})

In [10]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(149, 169, 136, 273, 107)

In [11]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

199.99473684210525

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [35]:
dict_temp = js.filter(lambda record: record['name'] == 'Alice').pluck('transactions')
total_temp = 0
# print(len(dict_temp))
for j in dict_temp:
    for i in j:
        total_temp += i['amount']
print(total_temp)

189422231


In [30]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 1219, 'amount': 4428},
 {'transaction-id': 1261, 'amount': 4314},
 {'transaction-id': 1486, 'amount': 4442})

In [14]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(4428, 4314, 4442)

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

996.9853469828154

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [16]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Edith', 'Alice', 'Frank'])]

In [17]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 7, 5, 9])]

In [18]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [19]:
b.compute()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [20]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [21]:
# %%time
# # Warning, this one takes a while...
# result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
# print(sorted(result))

In [22]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 950), ('Bob', 800), ('Charlie', 1450), ('Dan', 1350), ('Edith', 600), ('Frank', 943), ('George', 450), ('Hannah', 850), ('Ingrid', 947), ('Jerry', 750), ('Kevin', 950), ('Laura', 1250), ('Michael', 800), ('Norbert', 1150), ('Oliver', 700), ('Patricia', 747), ('Quinn', 1343), ('Ray', 950), ('Sarah', 850), ('Tim', 769), ('Ursula', 1550), ('Victor', 850), ('Wendy', 899), ('Xavier', 1050), ('Yvonne', 750), ('Zelda', 1208)]
CPU times: user 119 ms, sys: 33.1 ms, total: 152 ms
Wall time: 1.2 s


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [23]:
def sum_amounts(dict_account):
    total_amount = 0
#     print(dict_account)
    for transaction in dict_account['transactions']:
        total_amount += transaction['amount']
    return total_amount
# test_account = {'name': 'Alice', 'transactions': [{'amount': 55, 'id': 123}, {'amount': 2, 'id': 456}]}
# print(sum_amounts(test_account))
def sum_amounts_2(total, dict_account):
    return total + sum_amounts(dict_account)

57


In [25]:
%%time

# Your code here...
# from operator import add
# def incr(tot, _):
#     return tot + 1

result = js.foldby( key = 'name',
                    binop = sum_amounts_2,
                    initial = 0,
                    combine = add,
                    combine_initial = 0
                    ).compute()
print(type(result))
print(sorted(result))

<class 'list'>
[('Alice', 189422231), ('Bob', 94951479), ('Charlie', 697860612), ('Dan', 224694515), ('Edith', 126160192), ('Frank', 119725225), ('George', 62548677), ('Hannah', 341758976), ('Ingrid', 343351705), ('Jerry', 130511720), ('Kevin', 158024963), ('Laura', 358829965), ('Michael', 423516324), ('Norbert', 171530947), ('Oliver', 118243391), ('Patricia', 97736713), ('Quinn', 158994314), ('Ray', 218233872), ('Sarah', 194063338), ('Tim', 135482392), ('Ursula', 489219822), ('Victor', 170099377), ('Wendy', 207987370), ('Xavier', 252179742), ('Yvonne', 323468169), ('Zelda', 256321054)]
CPU times: user 132 ms, sys: 35 ms, total: 167 ms
Wall time: 1.32 s


## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [34]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Xavier,"[{'transaction-id': 6350, 'amount': 149}, {'tr..."
1,1,Jerry,"[{'transaction-id': 112, 'amount': 241}, {'tra..."
2,2,Ingrid,"[{'transaction-id': 611, 'amount': 7653}, {'tr..."
3,3,Patricia,"[{'transaction-id': 5283, 'amount': 318}, {'tr..."
4,4,Ursula,"[{'transaction-id': 2063, 'amount': 710}, {'tr..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [35]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 5.97 s, sys: 876 ms, total: 6.85 s
Wall time: 5.64 s


name
Alice       950
Bob         800
Charlie    1450
Dan        1350
Edith       600
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [32]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Xavier', 'amount': 149, 'transaction-id': 6350},
 {'id': 0, 'name': 'Xavier', 'amount': 118, 'transaction-id': 6786},
 {'id': 0, 'name': 'Xavier', 'amount': 132, 'transaction-id': 8133})

In [ ]:
df = transactions.to_dataframe()
df.head()

In [ ]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

### See also


*  [Blogpost analyzing github data on EC2](https://www.continuum.io/content/dask-distributed-and-anaconda-cluster)